In [1]:
# Add this cell at the beginning to ensure the latest versions
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml

In [3]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 8.9 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)

In [5]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)


# ✅ Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ✅ Step 1: Load and prepare dataset
# Load all four dataframes
df_train_v1 = pd.read_csv("/content/generated_prompts_50000_v2.csv")
df_train_v2 = pd.read_csv("/content/generated_prompts_50000_v3.csv")
df_train_v3 = pd.read_csv("/content/v1.csv")
df_train_v4 = pd.read_csv("/content/v2.csv")
df_train_v5 = pd.read_csv("/content/v3.csv")
df_train_v6 = pd.read_csv("/content/v4.csv", on_bad_lines='warn')


# Concatenate all four dataframes and remove duplicates
df_combined = pd.concat([df_train_v1, df_train_v2, df_train_v3, df_train_v4 , df_train_v5 ,df_train_v6], ignore_index=True)

# Inspect the combined dataframe before dropping duplicates and NaNs
print("Shape of combined dataframe before dropping duplicates and NaNs:", df_combined.shape)
display(df_combined.head())

df_combined = df_combined.drop_duplicates().dropna()

# Change all classifications that start with 'Code_' to 'Code'
df_combined.loc[df_combined['Classification'].str.startswith('Code_'), 'Classification'] = 'Code'
# Consolidate other 'Code' related classifications
df_combined.loc[df_combined['Classification'].isin(['Code  ', 'Code Explanation / Debug', 'Code Explanation / Debug"']), 'Classification'] = 'Code'


# Check the size of the combined dataframe after dropping duplicates and NaNs
print("Shape of combined dataframe after dropping duplicates and NaNs:", df_combined.shape)
if df_combined.shape[0] == 0:
    print("Error: The combined dataframe is empty after dropping duplicates and NaNs. Please check the input CSV files and the data cleaning steps.")
else:
    # Identify classes with only one sample
    class_counts = df_combined['Classification'].value_counts()
    classes_to_remove = class_counts[class_counts < 2].index

    # Remove rows belonging to classes with less than 2 samples
    df_combined = df_combined[~df_combined['Classification'].isin(classes_to_remove)]

    # Check the size again after removing small classes:
    print("Shape of combined dataframe after removing small classes:", df_combined.shape)

    if df_combined.shape[0] == 0:
         print("Error: The combined dataframe is empty after removing small classes. Please check the class distribution.")
    else:
        # Perform 80/20 train-test split on the combined data
        df_train, df_test = train_test_split(
            df_combined,
            test_size=0.2,
            random_state=SEED,
            stratify=df_combined['Classification'] # Stratify to maintain class distribution
        )


        # ✅ Step 2: Label encoding
        le = LabelEncoder()
        df_train["label"] = le.fit_transform(df_train["Classification"])
        df_test["label"] = le.transform(df_test["Classification"])

        # ✅ Step 3: Convert to Hugging Face Dataset
        train_ds = Dataset.from_pandas(df_train[["Prompt", "label"]])
        test_ds = Dataset.from_pandas(df_test[["Prompt", "label"]])

        print("Data loading, preprocessing, and splitting completed successfully.")

Shape of combined dataframe before dropping duplicates and NaNs: (157234, 2)


/tmp/ipython-input-5-1801962972.py:32: ParserWarning: Skipping line 8: expected 2 fields, saw 3
Skipping line 9: expected 2 fields, saw 3
Skipping line 17: expected 2 fields, saw 3
Skipping line 29: expected 2 fields, saw 3
Skipping line 38: expected 2 fields, saw 5
Skipping line 59: expected 2 fields, saw 3
Skipping line 71: expected 2 fields, saw 5
Skipping line 101: expected 2 fields, saw 5
Skipping line 122: expected 2 fields, saw 3
Skipping line 134: expected 2 fields, saw 4
Skipping line 143: expected 2 fields, saw 6
Skipping line 167: expected 2 fields, saw 5
Skipping line 188: expected 2 fields, saw 3
Skipping line 200: expected 2 fields, saw 5
Skipping line 224: expected 2 fields, saw 3
Skipping line 235: expected 2 fields, saw 3
Skipping line 236: expected 2 fields, saw 5
Skipping line 269: expected 2 fields, saw 5
Skipping line 290: expected 2 fields, saw 3
Skipping line 302: expected 2 fields, saw 5
Skipping line 323: expected 2 fields, saw 3
Skipping line 335: expected 2 f

,Prompt,Classification
0,Explain and debug this recursive memoization i...,Code Explanation / Debug
1,How would you fine-tune a BERT model for multi...,Code_aiml
2,What is the long-term impact of digital curren...,GK_economy
3,What experimental literary techniques did auth...,GK_literature
4,Write a sci-fi story where a hacker in North N...,Chat_story


Shape of combined dataframe after dropping duplicates and NaNs: (22949, 2)
Shape of combined dataframe after removing small classes: (22947, 2)
Data loading, preprocessing, and splitting completed successfully.


In [6]:
df_train.Classification.unique()

array(['GK_sports', 'Chat_story', 'Image Gen / Vision', 'GK_science"',
       'GK_environment', 'Classification', 'Chat_emotional_support',
       'Chat_advice', 'Chat_casual', 'GK_geography', 'GK_history',
       'GK_literature', 'GK_current_affairs"', 'GK_tech', 'Math',
       'Translation', 'Code', 'Chat_casual"', 'GK_current_affairs',
       'GK_economy"', 'Miscellaneous', 'GK_politics', 'Philosophy',
       'Chat_advice"', 'GK_science', 'Chat_emotional_support"',
       'GK_history"', 'Math"', 'GK_geography"', 'Astronomy',
       'Chat_story"', 'Art_Music', 'Summarization"', 'GK_tech"',
       'GK_literature"', 'Law_Politics', 'GK_sports"', 'GK_economy',
       'Summarization', 'Translation"', 'Health_Wellness', 'Linguistics',
       'GK_environment"', 'GK_politics"'], dtype=object)

In [7]:
df_test

,Prompt,Classification,label
151049,Why do people in British Indian Ocean Territor...,Chat_casual,4
47695,What tactical innovations led to Christopher M...,GK_sports,28
61514,What are the unique geological features of the...,GK_geography,18
153809,Write a story about a detective in Parsonston ...,Chat_story,8
154312,Tell a story about a robot exploring Lake Moll...,Chat_story,8
...,...,...,...
154379,Tell me a fun fact about daily life in Guyana.,Chat_casual,4
156535,Describe the evolution of **front-end testing ...,Code,11
50630,What tactical innovations led to Dylan Koch's ...,GK_sports,28
35658,Generate an image of a post-apocalyptic market...,Image Gen / Vision,33


In [8]:

# # ✅ Step 4: Load tokenizer and model using Auto classes
# model_name = "microsoft/MiniLM-L12-H384-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=len(le.classes_)
# )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:


# # ✅ Step 5: Tokenization function
# def tokenize(batch):
#     return tokenizer(
#         batch["Prompt"],
#         padding="max_length",
#         truncation=True,
#         max_length=128
#     )

# train_ds = train_ds.map(tokenize, batched=True, remove_columns=["Prompt"])
# test_ds = test_ds.map(tokenize, batched=True, remove_columns=["Prompt"])


Map:   0%|          | 0/18357 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Map:   0%|          | 0/4590 [00:00<?, ? examples/s]

In [10]:

# # ✅ Step 6: Metrics computation
# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = np.argmax(pred.predictions, axis=1)
#     precision, recall, f1, _ = precision_recall_fscore_support(
#         labels, preds, average='weighted', zero_division=0
#     )
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }

In [11]:
# # ✅ Step 7: Training arguments
# training_args = TrainingArguments(
#     output_dir="./minilm-model",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     num_train_epochs=7,  # Reduced epochs to 6
#     learning_rate=2e-6,
#     weight_decay=0.01,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     logging_dir="./logs",
#     logging_steps=100,
#     report_to="none",
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     greater_is_better=True,
#     save_total_limit=2,
#     fp16=torch.cuda.is_available(),
#     seed=SEED
# )

In [12]:

# # ✅ Step 8: Trainer setup
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     eval_dataset=test_ds,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )

/tmp/ipython-input-12-17687323.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:

# # ✅ Step 9: Train
# print("🚀 Starting training...")
# trainer.train()

# # ✅ Step 10: Save model and artifacts
# output_dir = "./minilm-model"
# os.makedirs(output_dir, exist_ok=True)

# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)
# pd.Series(le.classes_).to_csv(os.path.join(output_dir, "label_classes.csv"), index=False)

# print("✅ Training completed and model saved at", output_dir)


🚀 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,2.722100,2.618831,0.741394,0.656130,0.603190,0.741394
2,2.109000,2.023702,0.785621,0.706070,0.658394,0.785621
3,1.703100,1.649811,0.838780,0.786403,0.775201,0.838780
4,1.488000,1.415818,0.851416,0.803059,0.782100,0.851416
5,1.317400,1.272507,0.853813,0.807747,0.793212,0.853813
6,1.269100,1.195474,0.869499,0.835675,0.834333,0.869499
7,1.151600,1.170619,0.872331,0.839272,0.837782,0.872331


✅ Training completed and model saved at ./minilm-model


In [14]:
# import shutil
# shutil.make_archive('/content/my_model', 'zip', '/content/minilm-model')


'/content/my_model.zip'

In [ ]:
# from google.colab import files
# files.download('/content/my_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install sentence-transformers xgboost scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC  # ✅ Import SVM
# from sklearn.metrics import classification_report
# from sklearn.preprocessing import LabelEncoder
# from xgboost import XGBClassifier # Import XGBClassifier here


# # Convert raw text to Bag-of-Words vectors
# vectorizer = CountVectorizer(max_features=5000)
# X = vectorizer.fit_transform(df_train['Prompt'])
# y = df_train['Classification']

# # Encode string labels to integers
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# # Split into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
# )

# # --- 1. Logistic Regression ---
# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train, y_train)
# y_pred_lr = lr.predict(X_test)
# print("Logistic Regression:")
# print(classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_))

# # --- 2. K-Nearest Neighbors ---
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train, y_train)
# y_pred_knn = knn.predict(X_test)
# print("KNN:")
# print(classification_report(y_test, y_pred_knn, target_names=label_encoder.classes_))

# # --- 3. Decision Tree ---
# dt = DecisionTreeClassifier()
# dt.fit(X_train, y_train)
# y_pred_dt = dt.predict(X_test)
# print("Decision Tree:")
# print(classification_report(y_test, y_pred_dt, target_names=label_encoder.classes_))

# # --- 4. Support Vector Machine (SVM) ---
# svm = SVC(kernel='linear')  # You can change kernel to 'rbf', 'poly', etc.
# svm.fit(X_train, y_train)
# y_pred_svm = svm.predict(X_test)
# print("Support Vector Machine (SVM):")
# print(classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_))

# # --- 5. XGBoost Classifier ---
# xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  # ✅ For multiclass or binary
# xgb.fit(X_train, y_train)
# y_pred_xgb = xgb.predict(X_test)
# print("XGBoost:")
# print(classification_report(y_test, y_pred_xgb, target_names=label_encoder.classes_))

In [17]:
# from sklearn.feature_extraction.text import TfidfVectorizer  # ✅ TF-IDF instead of BoW
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC  # ✅ SVM
# from sklearn.metrics import classification_report
# from sklearn.preprocessing import LabelEncoder

# # Convert text to TF-IDF vectors
# vectorizer = TfidfVectorizer(max_features=5000)
# X = vectorizer.fit_transform(df_train['Prompt'])  # Text features
# y = df_train['Classification']  # Labels

# # Encode string labels into integers
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# # Split into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
# )

# # --- 1. Logistic Regression ---
# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train, y_train)
# y_pred_lr = lr.predict(X_test)
# print("Logistic Regression:")
# print(classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_))

# # --- 2. K-Nearest Neighbors ---
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train, y_train)
# y_pred_knn = knn.predict(X_test)
# print("KNN:")
# print(classification_report(y_test, y_pred_knn, target_names=label_encoder.classes_))

# # --- 3. Decision Tree ---
# dt = DecisionTreeClassifier()
# dt.fit(X_train, y_train)
# y_pred_dt = dt.predict(X_test)
# print("Decision Tree:")
# print(classification_report(y_test, y_pred_dt, target_names=label_encoder.classes_))

# # --- 4. Support Vector Machine (SVM) ---
# svm = SVC(kernel='linear')  # Best kernel for text data
# svm.fit(X_train, y_train)
# y_pred_svm = svm.predict(X_test)
# print("Support Vector Machine (SVM):")
# print(classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_))

# # --- 5. XGBoost ---
# xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# xgb.fit(X_train, y_train)
# y_pred_xgb = xgb.predict(X_test)
# print("XGBoost:")
# print(classification_report(y_test, y_pred_xgb, target_names=label_encoder.classes_))

In [18]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm

# --- 1. Load Model and Tokenizer ---
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()  # Inference mode

# --- 2. Encode Prompts into Embeddings ---
def get_embeddings(texts):
    embeddings = []
    with torch.no_grad():
        for text in tqdm(texts, desc="Embedding"):
            encoded = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
            output = model(**encoded)
            # Mean Pooling
            last_hidden = output.last_hidden_state
            attention_mask = encoded['attention_mask'].unsqueeze(-1).expand(last_hidden.size())
            summed = torch.sum(last_hidden * attention_mask, 1)
            summed_mask = torch.clamp(attention_mask.sum(1), min=1e-9)
            mean_pooled = summed / summed_mask
            embeddings.append(mean_pooled.squeeze().numpy())
    return np.array(embeddings)

# --- 3. Prepare Data ---
texts = df_train['Prompt'].tolist()
y = df_train['Classification'].tolist()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# # Convert texts to embeddings
# X = get_embeddings(texts)

# # --- 4. Train-Test Split ---
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
# )

# # --- 5. Logistic Regression ---
# lr = LogisticRegression(max_iter=1000)
# lr.fit(X_train, y_train)
# y_pred_lr = lr.predict(X_test)
# print("Logistic Regression:")
# print(classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_))

# # --- 6. K-Nearest Neighbors ---
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train, y_train)
# y_pred_knn = knn.predict(X_test)
# print("KNN:")
# print(classification_report(y_test, y_pred_knn, target_names=label_encoder.classes_))

# # --- 7. Decision Tree ---
# dt = DecisionTreeClassifier()
# dt.fit(X_train, y_train)
# y_pred_dt = dt.predict(X_test)
# print("Decision Tree:")
# print(classification_report(y_test, y_pred_dt, target_names=label_encoder.classes_))

# # --- 8. Support Vector Machine (SVM) ---
# svm = SVC(kernel='linear')
# svm.fit(X_train, y_train)
# y_pred_svm = svm.predict(X_test)
# print("Support Vector Machine (SVM):")
# print(classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_))


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [19]:
# Use this during training
X = get_embeddings(df_train["Prompt"].tolist())  # → shape (N, 384)
y = label_encoder.fit_transform(df_train["Classification"])

from sklearn.svm import SVC
svm = SVC(kernel="linear", probability=True)
svm.fit(X, y)

# Save the new model
import joblib
joblib.dump(svm, "svm_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")


Embedding: 100%|██████████| 18357/18357 [05:11<00:00, 58.88it/s]


['label_encoder.pkl']

In [22]:
# def extract_dummy_features(prompt):
#     return [
#         len(prompt),                    # prompt length
#         prompt.count(" "),             # number of words (roughly)
#         prompt.lower().startswith("write"),  # starts with "write"
#         int("def " in prompt),         # looks like code
#         int("story" in prompt.lower()),  # keyword signal
#     ]

In [ ]:
# import numpy as np
# X = np.array([extract_dummy_features(p) for p in df_train["Prompt"]])

# # Train/test split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, stratify=y, random_state=42
# )

# # Train model
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)

# # Predict
# y_pred = clf.predict(X_test)

# # Evaluate
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

                        precision    recall  f1-score   support

           Chat_advice       0.97      0.92      0.95        39
           Chat_casual       0.44      0.64      0.52        39
Chat_emotional_support       0.84      0.47      0.60       102
            Chat_story       1.00      1.00      1.00       528
    GK_current_affairs       0.96      0.67      0.79        39
        GK_environment       0.33      0.06      0.10       143
          GK_geography       0.91      0.88      0.90       363
            GK_history       0.66      0.83      0.74       120
         GK_literature       0.50      0.08      0.13        39
           GK_politics       0.30      0.18      0.23        39
             GK_sports       0.74      0.98      0.84       393
    Image Gen / Vision       0.92      1.00      0.95       685
           Translation       0.88      1.00      0.94        29

              accuracy                           0.86      2558
             macro avg       0.73     

In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report

# # Load your dataset
# df = df_train.copy() # Replace with actual file name
# from sklearn.utils import shuffle
# df = shuffle(df, random_state=42).reset_index(drop=True)

# # Extract prompts and labels
# X_text = df["Prompt"]
# y_labels = df["Classification"]

# # Encode labels numerically
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y_labels)

# # TF-IDF vectorizer (with unigrams + bigrams, max 10k features)
# vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
# X_tfidf = vectorizer.fit_transform(X_text)

# # Train/test split
# X_train, X_test, y_train, y_test = train_test_split(
#     X_tfidf, y, test_size=0.2, stratify=y, random_state=42
# )

# # Train Logistic Regression
# clf = LogisticRegression(max_iter=1000)
# clf.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = clf.predict(X_test)
# print("TF-IDF + Logistic Regression:")
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

TF-IDF + Logistic Regression:
                        precision    recall  f1-score   support

           Chat_advice       1.00      1.00      1.00        39
           Chat_casual       1.00      1.00      1.00        39
Chat_emotional_support       1.00      1.00      1.00       102
            Chat_story       1.00      1.00      1.00       528
    GK_current_affairs       1.00      1.00      1.00        39
        GK_environment       1.00      1.00      1.00       143
          GK_geography       1.00      1.00      1.00       363
            GK_history       1.00      1.00      1.00       120
         GK_literature       1.00      1.00      1.00        39
           GK_politics       1.00      1.00      1.00        39
             GK_sports       1.00      1.00      1.00       393
    Image Gen / Vision       1.00      1.00      1.00       685
           Translation       1.00      1.00      1.00        29

              accuracy                           1.00      2558
        

In [ ]:
# # Shuffle TRAIN labels only
# import numpy as np

# y_train_shuffled = np.random.permutation(y_train)
# clf.fit(X_train, y_train_shuffled)
# y_pred = clf.predict(X_test)  # <- NOT X_train!
# print("Accuracy on test set with shuffled labels:")
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy on test set with shuffled labels:
                        precision    recall  f1-score   support

           Chat_advice       0.00      0.00      0.00        39
           Chat_casual       0.00      0.00      0.00        39
Chat_emotional_support       0.00      0.00      0.00       102
            Chat_story       0.14      0.10      0.11       528
    GK_current_affairs       0.00      0.00      0.00        39
        GK_environment       0.00      0.00      0.00       143
          GK_geography       0.34      0.04      0.06       363
            GK_history       0.00      0.00      0.00       120
         GK_literature       0.00      0.00      0.00        39
           GK_politics       0.00      0.00      0.00        39
             GK_sports       0.24      0.03      0.06       393
    Image Gen / Vision       0.27      0.83      0.41       685
           Translation       0.00      0.00      0.00        29

              accuracy                           0.25      

In [ ]:
# # Count the occurrences of 'Code_devops' in the 'Classification' column of df_combined
# code_devops_count = df_combined[df_combined['Classification'] == 'Code'].value_counts()
# code_devops_count

,,count
Prompt,Classification,
"You suspect a **denial-of-service (DoS)** vulnerability in your public API. Outline your diagnostic process, including traffic analysis, rate limiting, and security testing.",Code,1
"A critical **data corruption** issue has been reported in a distributed data store. Describe your process for rapid diagnosis, data recovery, and preventing recurrence.",Code,1
"A critical **database deadlock** is occurring intermittently in your highly concurrent application. Describe your step-by-step debugging methodology, including transaction isolation levels and lock analysis.",Code,1
Adapt Dijkstra’s algorithm for weighted graphs in Python.,Code,1
Analyze memory usage patterns in a Java application,Code,1
...,...,...
"Build a Flask endpoint that accepts JSON payloads.""",Code,1
"Build a Flask API endpoint for user login.""",Code,1
Build a CSS framework with utility classes,Code,1
